In [309]:

import os
import csv
import re

units = {
    "s": 1.0,
    "ms": 0.001,
    "µs": 0.000001,
    "ns": 0.000000001,
    "mega_bytes": 1.0 / 1048576,  # 1 MB = 1024 * 1024 B
    "kilo_bytes": 1.0 / 1024  # 1 KB = 1024 B
}

batch = 4 # When n is small, the leader MSM occupies a big bottleneck, however, this can be efficiently optimized by parallel the batch MSM.
epsilon = 8 # Epsilon = N / k, where k is the PSS packing factor

def removeConsecutiveDuplicates(s):
    if len(s) < 2:
        return s
    if s[0] != s[1] or s[0] != '.':
        return s[0] + removeConsecutiveDuplicates(s[1:])
    return removeConsecutiveDuplicates(s[1:])

def splitString(str):
    alpha = ""
    num = ""
    for i in range(len(str)):
        if (str[i].isdigit() or str[i] == '.'):
            num = num + str[i]
        else:
            alpha += str[i]  
    return alpha, num

In [310]:
# zkSaaS Collaborative Plonk

data = []

d_plonk_operations = [
    ("FFT1", "local"),
    ("Compute r", "local"),
    ("Compute t", "local"),
    ("Local DPP", "local"),
    ("DppRand", "local"),
    ("Division", "local"),
    ("FFT2", "king"),
    ("DPP", "king"),
    ("ComToKing", "comm"),
    ("ComFromKing", "comm"),
    ("Opening shares", "king"),
    ("Packing shares", "king"),
    ("Unpack Pack shares", "king"),
    ("Base MSM", "msm")
]

dir = 'zkSaaS_8'

# Read leader server files
for filename in os.listdir(dir):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'{dir}/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'k': None, 'time(s)': 0, 'mem(MB)': 0, 'comm(MB)': 0, 'local': 0, 'king': 0, "comm": 0, "msm": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][0])
            record['k'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][1])
            record['N'] = record['k'] * epsilon
            
            for line in content:
                for op_name, op_var in d_plonk_operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms

                if 'bytes_sent:' in line:
                    bytes_sent = int(re.search(r'bytes_sent: (\d+)', line).group(1))
                    record['comm(MB)'] += bytes_sent * units['mega_bytes']

                if 'bytes_recv:' in line:
                    bytes_recv = int(re.search(r'bytes_recv: (\d+)', line).group(1))
                    record['comm(MB)'] += bytes_recv * units['mega_bytes']
                    
                if 'Maximum resident set size' in line:
                    mem = int(re.search(r'(\d+)', line).group(0))
                    record['mem(MB)'] = mem * units['kilo_bytes']

             # Calculate total time
            record['time(s)'] = record['local'] + record['king'] + record['comm'] + record['msm']
            
            data.append(record)

# Sort data
data = sorted(data, key=lambda x: x['N'])
data = sorted(data, key=lambda x: x['k'])
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = f'{dir}_leader.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'k', 'N', 'time(s)', 'mem(MB)', 'comm(MB)', 'local','king', "comm", "msm"])
    writer.writeheader()
    for row in data:
        writer.writerow(row)
        
data = []

# Read worker server files
for filename in os.listdir(dir):
    if filename.startswith('log_1_') and filename.endswith('.txt'):
        with open(f'{dir}/{filename}', 'r') as file:
            content = file.readlines()

            record = {'n': None, 'k': None, 'time(s)': 0, 'mem(MB)': 0, 'comm(MB)': 0, 'local': 0, 'king': 0, "comm": 0, "msm": 0}
            record['n'] = int(re.findall(r'log_1_(\d+)_(\d+)', filename)[0][0])
            record['k'] = int(re.findall(r'log_1_(\d+)_(\d+)', filename)[0][1])
            record['N'] = record['k'] * epsilon
            
            for line in content:
                for op_name, op_var in d_plonk_operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms
                    
                if 'Maximum resident set size' in line:
                    mem = int(re.search(r'(\d+)', line).group(0))
                    record['mem(MB)'] = mem * units['kilo_bytes']

             # Calculate total time
            record['time(s)'] = record['local'] + record['king'] + record['comm'] + record['msm']
            
            data.append(record)

# Sort data
data = sorted(data, key=lambda x: x['N'])
data = sorted(data, key=lambda x: x['k'])
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = f'{dir}_worker.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'k', 'N', 'time(s)', 'mem(MB)', 'comm(MB)', 'local','king', "comm", "msm"])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [311]:
# local Hyperplonk
data = []

operations = [
    ("Commit", "poly_comm"),
    ("Gate identity", "gate"),
    ("Wire identity", "wire"),
    ("Open", "poly_comm"),
]

dir = 'hyperplonk'

# Read files
for filename in os.listdir(dir):
    if filename.startswith(f'{dir}_') and filename.endswith('.txt'):
        with open(f'{dir}/{filename}', 'r') as file:
            content = file.readlines()

            record = {'time(s)': 0, 'mem(MB)': 0, 'poly_comm': 0, "gate": 0, "wire": 0}
            record['n'] = int(re.findall(fr'{dir}_(\d+)', filename)[0])
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms
                
                if 'Local HyperPlonk (thread ThreadId(1))' in line:
                    matches = re.search(r'(\d+\.\d+)(\D+)', line)
                    if matches:
                        alpha, num = splitString(matches.group(0))
                        time_in_s = float(num) * units.get(alpha.strip(), 1)
                        record['time(s)'] = time_in_s         
                        
                if 'maxresident' in line:
                    matches = re.findall(r'(\d+)maxresident', line)
                    if matches:
                        mem = int(matches[0])
                        record['mem(MB)'] = mem * units['kilo_bytes']

            # Calculate total time
            record['time(s)'] = min(record['time(s)'], (record['gate'] + record['wire'] + record['poly_comm']))
            
            data.append(record)
            
# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = f'{dir}.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'time(s)', 'mem(MB)', 'gate', 'wire', 'poly_comm'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [312]:
# local Hyperplonk++
data = []

operations = [
    ("Commit", "poly_comm"),
    ("Gate identity", "gate"),
    ("Wire identity", "wire"),
    ("Open", "poly_comm"),
]

dir = 'hyperplonkpp'

# Read files
for filename in os.listdir(dir):
    if filename.startswith(f'{dir}_') and filename.endswith('.txt'):
        with open(f'{dir}/{filename}', 'r') as file:
            content = file.readlines()

            record = {'time(s)': 0, 'mem(MB)': 0, 'poly_comm': 0, "gate": 0, "wire": 0}
            record['n'] = int(re.findall(fr'{dir}_(\d+)', filename)[0])
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_ms = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_ms
                
                if 'Local HyperPlonk++ (thread ThreadId(1))' in line:
                    matches = re.search(r'(\d+\.\d+)(\D+)', line)
                    if matches:
                        alpha, num = splitString(matches.group(0))
                        time_in_s = float(num) * units.get(alpha.strip(), 1)
                        record['time(s)'] = time_in_s         
                        
                if 'maxresident' in line:
                    matches = re.findall(r'(\d+)maxresident', line)
                    if matches:
                        mem = int(matches[0])
                        record['mem(MB)'] = mem * units['kilo_bytes']

            # Calculate total time
            record['time(s)'] = min(record['time(s)'], (record['gate'] + record['wire'] + record['poly_comm']))
            
            data.append(record)
            
# Sort data by 'n' in ascending order
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = f'{dir}.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'time(s)', 'mem(MB)', 'gate', 'wire', 'poly_comm'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [313]:
# Collaborative Hyperplonk++ 
data = []

operations = [
    ("Comm:", "Comm"),
    ("Local:", "Local"),
    ("Leader:", "Leader"),
    ("Leader: Compute element (MSM Leader)", "LeaderMSM"),
]

dir = 'dhyperplonk'

# Read files
for filename in os.listdir(dir):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'{dir}/{filename}', 'r') as file:
            content = file.readlines()

            record = {'time(s)': 0, 'mem(MB)': 0, 'comm(MB)': 0, 'Comm': 0, "Local": 0, "Leader": 0, "LeaderMSM": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][0])
            record['k'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][1])
            record['N'] = record['k'] * epsilon
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_s = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_s
                            
                if 'Distributed HyperPlonk (thread ThreadId(1))' in line:
                    matches = re.search(r'(\d+\.\d+)(\D+)', line)
                    if matches:
                        alpha, num = splitString(matches.group(0))
                        time_in_s = float(num) * units.get(alpha.strip(), 1)
                        record['time(s)'] = time_in_s         

                if 'Comm:' in line:
                    matches = re.findall(r'(\d+), (\d+)', line)
                    if matches:
                        comm = sum(map(int, matches[0]))
                        record['comm(MB)'] = comm * units['mega_bytes']
                        
                if 'Maximum resident' in line:
                    matches = re.findall(r'(\d+)', line)
                    if matches:
                        mem = int(matches[0])
                        record['mem(MB)'] = mem * units['kilo_bytes']

             # Calculate total time, smaller one of the two
            record['time(s)'] = min(record['time(s)'], record['Comm'] + record['Local'] + record['Leader'] - record['LeaderMSM'] + record['LeaderMSM'] / batch)
            
            data.append(record)

# Sort data
data = sorted(data, key=lambda x: x['N'])
data = sorted(data, key=lambda x: x['k'])
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = f'{dir}.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'k', 'N', 'time(s)', 'mem(MB)', 'comm(MB)', 'Comm', 'Local', 'Leader', 'LeaderMSM'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [314]:
# Collaborative Hyperplonk++ Data-parallel 

data = []

operations = [
    ("Comm:", "Comm"),
    ("Local:", "Local"),
    ("Leader:", "Leader"),
    ("Leader: Compute element (MSM Leader)", "LeaderMSM"),
]

dir = 'dhyperplonk-dp'

# Read files
for filename in os.listdir(dir):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'{dir}/{filename}', 'r') as file:
            content = file.readlines()

            record = {'time(s)': 0, 'mem(MB)': 0, 'comm(MB)': 0, 'Comm': 0, "Local": 0, "Leader": 0, "LeaderMSM": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][0])
            record['k'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][1])
            record['N'] = record['k'] * epsilon
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_s = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_s
                            
                if 'Distributed HyperPlonk (thread ThreadId(1))' in line:
                    matches = re.search(r'(\d+\.\d+)(\D+)', line)
                    if matches:
                        alpha, num = splitString(matches.group(0))
                        time_in_s = float(num) * units.get(alpha.strip(), 1)
                        record['time(s)'] = time_in_s         

                if 'Comm:' in line:
                    matches = re.findall(r'(\d+), (\d+)', line)
                    if matches:
                        comm = sum(map(int, matches[0]))
                        record['comm(MB)'] = comm * units['mega_bytes']
                        
                if 'Maximum resident' in line:
                    matches = re.findall(r'(\d+)', line)
                    if matches:
                        mem = int(matches[0])
                        record['mem(MB)'] = mem * units['kilo_bytes']

             # Calculate total time, smaller one of the two
            record['time(s)'] = min(record['time(s)'], record['Comm'] + record['Local'] + record['Leader'] - record['LeaderMSM'] + record['LeaderMSM'] / batch)
            
            data.append(record)

# Sort data
data = sorted(data, key=lambda x: x['N'])
data = sorted(data, key=lambda x: x['k'])
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = f'{dir}.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'k', 'N', 'time(s)', 'mem(MB)', 'comm(MB)', 'Comm', 'Local', 'Leader', 'LeaderMSM'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [315]:
# Distributed permcheck

data = []

operations = [
    ("Comm:", "Comm"),
    ("Local:", "Local"),
    ("Leader:", "Leader"),
    ("Leader: Compute element (MSM Leader)", "LeaderMSM"),
]

dir = 'dpermcheck'

# Read files
for filename in os.listdir(dir):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'{dir}/{filename}', 'r') as file:
            content = file.readlines()

            record = {'time(s)': 0, 'mem(MB)': 0, 'comm(MB)': 0, 'Comm': 0, "Local": 0, "Leader": 0, "LeaderMSM": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][0])
            record['k'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][1])
            record['N'] = record['k'] * epsilon
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_s = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_s
                            
                if 'Distributed Permcheck (thread ThreadId(1))' in line:
                    matches = re.search(r'(\d+\.\d+)(\D+)', line)
                    if matches:
                        alpha, num = splitString(matches.group(0))
                        time_in_s = float(num) * units.get(alpha.strip(), 1)
                        record['time(s)'] = time_in_s         

                if 'Comm:' in line:
                    matches = re.findall(r'(\d+), (\d+)', line)
                    if matches:
                        comm = sum(map(int, matches[0]))
                        record['comm(MB)'] = comm * units['mega_bytes']
                        
                if 'Maximum resident' in line:
                    matches = re.findall(r'(\d+)', line)
                    if matches:
                        mem = int(matches[0])
                        record['mem(MB)'] = mem * units['kilo_bytes']

             # Calculate total time, smaller one of the two
            record['time(s)'] = min(record['time(s)'], record['Comm'] + record['Local'] + record['Leader'] - record['LeaderMSM'] + record['LeaderMSM'] / batch)
            
            data.append(record)

# Sort data
data = sorted(data, key=lambda x: x['N'])
data = sorted(data, key=lambda x: x['k'])
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = f'{dir}.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'k', 'N', 'time(s)', 'mem(MB)', 'comm(MB)', 'Comm', 'Local', 'Leader', 'LeaderMSM'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)

In [316]:
# Collaborative permcheck

data = []

operations = [
    ("Comm:", "Comm"),
    ("Local:", "Local"),
    ("Leader:", "Leader"),
    ("Leader: Compute element (MSM Leader)", "LeaderMSM"),
]

dir = 'cpermcheck'

# Read files
for filename in os.listdir(dir):
    if filename.startswith('log_0') and filename.endswith('.txt'):
        with open(f'{dir}/{filename}', 'r') as file:
            content = file.readlines()

            record = {'time(s)': 0, 'mem(MB)': 0, 'comm(MB)': 0, 'Comm': 0, "Local": 0, "Leader": 0, "LeaderMSM": 0}
            record['n'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][0])
            record['k'] = int(re.findall(r'log_0_(\d+)_(\d+)', filename)[0][1])
            record['N'] = record['k'] * epsilon
            
            for line in content:
                for op_name, op_var in operations:
                    if 'End:' in line and op_name in line:
                        cleaned_line = removeConsecutiveDuplicates(line)
                        time_string = re.search(r'(\d+\.\d+)(\D+)', cleaned_line)
                        if time_string:
                            alpha, num = splitString(time_string.group(0))
                            time_in_s = float(num) * units.get(alpha.strip(), 1)
                            record[op_var] = record.get(op_var, 0) + time_in_s
                            
                if 'Collaborative Permcheck (thread ThreadId(1))' in line:
                    matches = re.search(r'(\d+\.\d+)(\D+)', line)
                    if matches:
                        alpha, num = splitString(matches.group(0))
                        time_in_s = float(num) * units.get(alpha.strip(), 1)
                        record['time(s)'] = time_in_s         

                if 'Comm:' in line:
                    matches = re.findall(r'(\d+), (\d+)', line)
                    if matches:
                        comm = sum(map(int, matches[0]))
                        record['comm(MB)'] = comm * units['mega_bytes']
                        
                if 'Maximum resident' in line:
                    matches = re.findall(r'(\d+)', line)
                    if matches:
                        mem = int(matches[0])
                        record['mem(MB)'] = mem * units['kilo_bytes']

             # Calculate total time, smaller one of the two
            record['time(s)'] = min(record['time(s)'], record['Comm'] + record['Local'] + record['Leader'] - record['LeaderMSM'] + record['LeaderMSM'] / batch)
            
            data.append(record)

# Sort data
data = sorted(data, key=lambda x: x['N'])
data = sorted(data, key=lambda x: x['k'])
data = sorted(data, key=lambda x: x['n'])

# Save to .csv
csv_file = f'{dir}.csv'
with open(csv_file, 'w', newline='') as file:
    writer = csv.DictWriter(file, fieldnames=['n', 'k', 'N', 'time(s)', 'mem(MB)', 'comm(MB)', 'Comm', 'Local', 'Leader', 'LeaderMSM'])
    writer.writeheader()
    for row in data:
        writer.writerow(row)